In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
!pip install torchsummary


In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 10, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(10)
        self.pool1 = nn.MaxPool2d(2, 2)

        # Block 2
        self.conv3 = nn.Conv2d(10, 16, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 18, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(18)
        self.pool2 = nn.MaxPool2d(2, 2)

        # Block 3 (increased width)
        self.conv5 = nn.Conv2d(18, 32, 3)     # 7->5
        self.bn5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 24, 3)     # 5->3
        self.bn6 = nn.BatchNorm2d(24)
        self.conv7 = nn.Conv2d(24, 10, 3)     # 3->1

        self.dropout = nn.Dropout2d(0.25)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)

        # Block 2
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool2(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout(x)

        # Block 3
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.conv7(x)

        # GAP
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(-1, 10)

        return F.log_softmax(x, dim=1)


In [19]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]              60
       BatchNorm2d-2            [-1, 6, 28, 28]              12
            Conv2d-3           [-1, 10, 28, 28]             550
       BatchNorm2d-4           [-1, 10, 28, 28]              20
         MaxPool2d-5           [-1, 10, 14, 14]               0
         Dropout2d-6           [-1, 10, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,456
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 18, 14, 14]           2,610
      BatchNorm2d-10           [-1, 18, 14, 14]              36
        MaxPool2d-11             [-1, 18, 7, 7]               0
        Dropout2d-12             [-1, 18, 7, 7]               0
           Conv2d-13             [-1, 32, 5, 5]           5,216
      BatchNorm2d-14             [-1, 3

In [35]:
print(device)

cuda


In [12]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 477kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.0MB/s]


In [13]:
train_size = len(train_loader.dataset)
test_size = len(test_loader.dataset)
total_size = train_size + test_size

print(f"Dataset sizes:")
print(f"  Training: {train_size:,} samples")
print(f"  Test: {test_size:,} samples")
print(f"  Total: {total_size:,} samples")
print(f"  Train/Test split: {train_size/total_size:.1%}/{test_size/total_size:.1%}")
print(f"\nBatch info:")
print(f"  Batch size: {batch_size}")
print(f"  Training batches: {len(train_loader)}")
print(f"  Test batches: {len(test_loader)}")

Dataset sizes:
  Training: 60,000 samples
  Test: 10,000 samples
  Total: 70,000 samples
  Train/Test split: 85.7%/14.3%

Batch info:
  Batch size: 128
  Training batches: 469
  Test batches: 79


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [17]:

model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(1, 15):
    print(f"Epoch: {epoch}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()  # <-- move LR schedule forward


Epoch: 1


loss=0.08483710139989853 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 29.27it/s]



Test set: Average loss: 0.0541, Accuracy: 9826/10000 (98.26%)

Epoch: 2


loss=0.021504759788513184 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.21it/s]



Test set: Average loss: 0.0376, Accuracy: 9873/10000 (98.73%)

Epoch: 3


...


Test set: Average loss: 0.0177, Accuracy: 9932/10000 (99.32%)

Epoch: 13


loss=0.04223646596074104 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.10it/s]



Test set: Average loss: 0.0180, Accuracy: 9940/10000 (99.40%)

Epoch: 14


loss=0.015097707509994507 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.19it/s]



Test set: Average loss: 0.0170, Accuracy: 9943/10000 (99.43%)

